## Importing libraries

In [83]:
import subprocess 
import json
import os
from dotenv import load_dotenv
from constants import *
from bit import Key, PrivateKey, PrivateKeyTestnet
from bit.network import NetworkAPI
from bit import *
from web3 import Web3
from eth_account import Account 

## Web3 connection and loading mnemonic. Nodes with POW

In [85]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1.8545"))

# Loading EV
load_dotenv

# Loading Mnemonic EV
mnemonic = os.getenv('MNEMONIC')

## Functions to transact

In [73]:
def derive_wallets(mnemonic, coin, numderive):
    """Use the subprocess library to call the ./derive php file script from Python"""
    command = 'php ./hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="'+str(mnemonic)+'" --numderive='+str(numderive)+' --coin='+str(coin)+' --format=jsonpretty' 
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    return json.loads(output) 

# Create a coin object to hold child wallets
coins = {'eth':derive_wallets(mnemonic=mnemonic,coin=ETH,numderive=3),'btc-test': derive_wallets(mnemonic=mnemonic,coin=BTCTEST,numderive=3)}

eth_privatekey = coins['eth'][0]['privkey']
btc_privatekey = coins['btc-test'][0]['privkey']

In [74]:
print(json.dumps(coins, indent=4, sort_keys=True))

{
    "btc-test": [
        {
            "address": "msQvwuLLPSgh6WUqn24HmNm5CyuEF7uhpv",
            "index": 0,
            "path": "m/44'/1'/0'/0/0",
            "privkey": "cQr2vvFf419mpWRYuiCCXQWuKF5xg5qYtPnUGRmzun2f9jVX8FSg",
            "pubkey": "0254181993bf8e25e782655f55dca3905c52ae55463a4483e46d5e5632c3107f11",
            "pubkeyhash": "827de38c24d36cc100535e41b6e6e014f0e01857",
            "xprv": "tprv8jP5BA39mrSonCGAgUKF4m4QhHiiRA78aX4EVEPKbMafKPCu7J3os2WsYbxXU8cxzBvpvKCCnKfBK7Gz2PVrQ27QNCb1oeBtGuToVpG1u6q",
            "xpub": "tpubDG57Ka5PvE8UffHxa7yqUAiXGKEeaVJ39pf1mkRd1dP49sTfjgsQ3X8jiioZZAfvVvvgKSZFUAAKEPgzaVMmaDwJSfrT9hSWX6Qw9M3cNZo"
        },
        {
            "address": "muSU9Bhyot94s6jYc8KJMsCZvTLN5hMGpD",
            "index": 1,
            "path": "m/44'/1'/0'/0/1",
            "privkey": "cNzE283VyW3KMox9gMtDt5X8FYncvgSMJv66GDdGx9XnNRoPRXir",
            "pubkey": "0308cc5690172e500c36045d281a3472f83fc422317e0ff9a11caf5d7915153b77",
            "pubkeyh

In [75]:
def priv_key_to_account(coin, priv_key):
    """Convert the privkey string in a child key to an account object that bit or web3.py can use to transact"""
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)
    
eth_account = priv_key_to_account(ETH,eth_privatekey)
btc_account = priv_key_to_account(BTCTEST,btc_privatekey)

In [76]:
print(eth_account)
print(btc_account)

<PrivateKeyTestnet: msQvwuLLPSgh6WUqn24HmNm5CyuEF7uhpv>


In [80]:
def create_tx(coin, account, recipient, amount):
    """create the raw, unsigned transaction that contains all metadata needed to transact"""
    global tx_data
    if coin ==ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )
        tx_data = {
            "to": recipient,
            "from": account.address,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address)
        }
        return tx_data
        
    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])   

In [89]:
create_tx(BTCTEST,btc_account,'msQvwuLLPSgh6WUqn24HmNm5CyuEF7uhpv',0.0001)

'{"unspents":[{"amount":1768415,"confirmations":5210,"script":"76a914827de38c24d36cc100535e41b6e6e014f0e0185788ac","txid":"6f6165765c5d39246564b5cb5c7b45116b9dd72acd57e298d7a0e929a98d6dbb","txindex":1,"type":"p2pkh","vsize":148,"segwit":false}],"outputs":[["msQvwuLLPSgh6WUqn24HmNm5CyuEF7uhpv",10000],["msQvwuLLPSgh6WUqn24HmNm5CyuEF7uhpv",1754347]]}'

In [91]:
def send_tx(coin, account, recipient, amount):
    """call create_tx, sign the transaction, then send it to the designated network"""
    if coin =='ETH':
        tx_eth = create_tx(coin,account, recipient, amount)
        sign_tx_eth = account.sign_transaction(tx_eth)
        result = w3.eth.sendRawTransaction(sign_tx_eth.rawTransaction)
        print(result.hex())
        return result.hex()
    else:
        tx_btctest= create_tx(coin,account,recipient,amount)
        sign_tx_btctest = account.sign_transaction(tx_btctest)
        from bit.network import NetworkAPI
        NetworkAPI.broadcast_tx_testnet(sign_tx_btctest)       
        return sign_tx_btctest

In [92]:
send_tx(BTCTEST,btc_account,'msQvwuLLPSgh6WUqn24HmNm5CyuEF7uhpv',0.0001)

'0100000001bb6d8da929e9a0d798e257cd2ad79d6b11457b5ccbb5646524395d5c7665616f010000006b483045022100a04cceedabe288209b3629133398d3dd1f6db526e0d3e1477ee71dac555e66b5022072c94224cd7aa1f22e726274524cf1b3a71e6664e8f1a4b7a80c508a73c4149501210254181993bf8e25e782655f55dca3905c52ae55463a4483e46d5e5632c3107f11ffffffff0210270000000000001976a914827de38c24d36cc100535e41b6e6e014f0e0185788acebc41a00000000001976a914827de38c24d36cc100535e41b6e6e014f0e0185788ac00000000'